In [7]:
import torch
import gc
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import warnings
warnings.filterwarnings('ignore', message='.*safety checker.*')

def Diffusion(img_path):
    if not hasattr(Diffusion, 'pipe'):
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            'runwayml/stable-diffusion-v1-5',
            dtype=torch.float16,
            cache_dir='/opt/dlami/nvme/huggingface',
            safety_checker=None
        )
        pipe.to('cuda:0' if torch.cuda.is_available() else 'cpu')
        pipe.enable_vae_tiling()
        # pipe.enable_xformers_memory_efficient_attention(),
        Diffusion.pipe = pipe

    pipe = Diffusion.pipe
    init_image = Image.open(img_path).convert("RGB")

    with torch.inference_mode():
        generator = torch.Generator(device='cuda').manual_seed(torch.randint(0, 2 ** 32 - 1, (1,)).item())
        generated = pipe(
            prompt="An ancient glass bead excavated in Japan, smooth surface, realistic, subtle imperfections, natural color",
            negative_prompt="clustered particles, bumpy surface, multiple beads, noise, abstract",
            image=init_image,
            strength=0.45,
            guidance_scale=5.5,
            num_inference_steps=25,
            generator=generator
        ).images[0]
    generated.save('Data/1564_gen.jpg')

    torch.cuda.empty_cache()
    gc.collect()

Diffusion('Data/Few_Sample/Top/mix-Al/1564.jpg')

Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionImg2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
import os
import torch
import gc
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="diffusers")


def Diffusion(img_path, save_path):

    if not hasattr(Diffusion, 'pipe'):
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            'runwayml/stable-diffusion-v1-5',
            torch_dtype=torch.float16,          # ← 正しい引数名
            cache_dir='/opt/dlami/nvme/huggingface',
            safety_checker=None
        )

        # テキストエンコーダは CPU に残す（メモリ節約 約2GB）
        pipe.text_encoder.to("cpu")

        # UNet・VAE は GPU に
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        pipe.to(device)

        # メモリ効率化
        pipe.vae.enable_tiling()
        pipe.enable_vae_tiling()
        try:
            pipe.enable_xformers_memory_efficient_attention()
        except:
            pass

        Diffusion.pipe = pipe

    pipe = Diffusion.pipe

    init_image = Image.open(img_path).convert("RGB")

    with torch.inference_mode():
        generator = torch.Generator(device='cuda' if torch.cuda.is_available() else 'cpu').manual_seed(
            torch.randint(0, 2**32 - 1, (1,)).item()
        )

        generated = pipe(
            prompt="An ancient glass bead excavated in Japan, smooth surface, realistic, subtle imperfections, natural color",
            negative_prompt="clustered particles, bumpy surface, multiple beads, noise, abstract",
            image=init_image,
            strength=0.45,
            guidance_scale=5.5,
            num_inference_steps=15,
            generator=generator
        ).images[0]

    generated.save(save_path)

    # メモリ解放
    del generator
    torch.cuda.empty_cache()
    gc.collect()



# =======================================
#  一括処理
# =======================================

name = 'LI'

input_dir  = f"Data/Few_Sample/Top/{name}/"
output_dir = f"Data/Diffusion/Top/{name}/"

os.makedirs(output_dir, exist_ok=True)

valid_ext = (".jpg", ".png", ".jpeg", ".bmp", ".webp")
files = [f for f in os.listdir(input_dir) if f.lower().endswith(valid_ext)]

print(f"{len(files)} 枚の画像を処理します。")

for fname in files:
    img_path  = os.path.join(input_dir, fname)
    save_path = os.path.join(output_dir, fname)

    print(f"生成中: {fname}...")
    Diffusion(img_path, save_path)

print("完了しました！")

14 枚の画像を処理します。
生成中: 4109.jpg...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/6 [00:00<?, ?it/s]

In [1]:
import torch
import torchvision

print('torch', torch.__version__)
print('torchvision', torchvision.__version__)

torch 2.8.0+cu129
torchvision 0.23.0+cu129
